In [1]:
# Тюнинг
import optuna as opt
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import BaggingClassifier, StackingClassifier
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.feature_selection import RFE

# Пайплайн
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

# Данные
import os
import pandas as pd
import numpy as np
from category_encoders import BinaryEncoder, OneHotEncoder
from sklearn.preprocessing import RobustScaler

In [3]:
# Пути
ROOT = os.getcwd()
TRAIN_DATASET = os.path.join(ROOT, '../data/train_AIC.csv')
BALANCED_DATASET = os.path.join(ROOT, '../data/balanced_train.csv')
TEST_DATASET = os.path.join(ROOT, '../data/test_AIC.csv')
SUBMISSION_PATH = os.path.join(ROOT, '../submissions/')

def save_submission(model, subname):
    subname = os.path.join(SUBMISSION_PATH, f'{subname}.csv')
    preds = model.predict(test_df)
    submit_df = pd.DataFrame({'id': test_df.index, 'value': preds})
    submit_df.to_csv(subname, index=False)

# Загрузка
train_df = pd.read_csv(TRAIN_DATASET)
# balanced_df = pd.read_csv(BALANCED_DATASET, index_col=0)
test_df = pd.read_csv(TEST_DATASET)

# first_negatives = train_df[train_df['y'] == 0][:train_df[train_df['y'] == 1]['y'].count()]
# train_df = pd.concat([train_df[train_df['y'] == 1], first_negatives])

# def random_undersample(df):
#     neg_count, pos_count = np.bincount(df['y'])
#     pos_df = df[df['y'] == 1]
#     neg_df = df[df['y'] == 0]
#     neg_df = neg_df.sample(n=pos_count, random_state=1708)
#     return pd.concat([pos_df, neg_df])

# balanced_df = random_undersample(train_df)

X, y = train_df.iloc[:, :-1], train_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

FEATURES_TO_DROP = [
    'Change on Paper', 'Amount', 'Category Manager', 'Factory', 
    'Material', 'Cansel Complete Release', 'Approval Cycles'
    ]

# FEATURES_TO_KEEP = ['Поставщик', 'Длительность', 'Закупочная организация', 'Балансовая единица',
#                     'ЕИ', 'Сумма', 'Вариант поставки', 'НРП', 'Вариант поставки',
#                     'Месяц1', 'Месяц2', 'Месяц3', 'День недели 2']
# FEATURES_TO_DROP = X.columns[~X.columns.isin(FEATURES_TO_KEEP)]

X_train = X_train.drop(FEATURES_TO_DROP, axis=1)
X_test = X_test.drop(FEATURES_TO_DROP, axis=1)
test_df = test_df.drop(FEATURES_TO_DROP, axis=1)

NameError: name 'pd' is not defined

In [3]:
# Препроцессоры
class DataPreprocessor(BaseEstimator, TransformerMixin):
    """ Предобработчик данных """
    def __init__(self, cat_features, transform_train=True):
        self.transform_train = transform_train
        self.cat_features = cat_features

        self.bin_encoder = BinaryEncoder(cols=cat_features)
        self.onehot_encoder = OneHotEncoder(cols=['Поставщик'])
        self.robust_scaler = RobustScaler()
        self.rename_cols = [
            'Provider', 'Material', 'Category Manager', 'Operations Manager',
            'Factory', 'Purchasing Organization', 'Purchasing Group', 
            'Company Code', 'EI', 'Material Group', 'Delivery Option', 'NRP',
            'Duration', 'ETC Delivery', 'Month1', 'Month2', 'Month3', 'Weekday',
            'Sum', 'Position Count', 'Amount', 'Handlers 7', 'Handlers 15', 
            'Handlers 30', 'Order Approval 1', 'Order Approval 2', 'Order Approval 3',
            'Change Delivery Date 7', 'Change Delivery Date 15', 'Change Delivery Date 30',
            'Cancel Complete Release', 'Change on Paper',
            'Delivery Date', 'Approval Cycles', 'Changes After Approvals', 
            'Days Between 0_1', 'Days Between 1_2', 'Days Between 2_3',
            'Days Between 3_4', 'Days Between 4_5', 'Days Between 5_6',
            'Days Between 6_7', 'Days Between 7_8' 
            ]

    def fit(self, X, y=None):
        # Создаём копию датасета
        X_ = X.copy()
        X_.columns = self.rename_cols

        # Временные фичи
        X_['Weekday'] += 1
        X_['day_sin'] = np.sin(np.pi * 2 * X_['Weekday'] / 7)
        X_['day_cos'] = np.cos(np.pi * 2 * X_['Weekday'] / 7)
        X_['month1_sin'] = np.sin(np.pi * 2 * X_['Month1'] / 12)
        X_['month1_cos'] = np.cos(np.pi * 2 * X_['Month1'] / 12)
        X_['month2_sin'] = np.sin(np.pi * 2 * X_['Month2'] / 12)
        X_['month2_cos'] = np.cos(np.pi * 2 * X_['Month2'] / 12)
        X_['month3_sin'] = np.sin(np.pi * 2 * X_['Month3'] / 12)
        X_['month3_cos'] = np.cos(np.pi * 2 * X_['Month3'] / 12)

        # Категориальные фичи
        X_ = self.bin_encoder.fit_transform(X_)

        features_to_drop = ('Weekday', 'Month1', 'Month2', 'Month3')
        X_ = X_.drop(features_to_drop, axis=1)

        # Масштабирование
        # self.robust_scaler.fit(X_)

        return self
    
    def transform(self, X):
        # Создаём копию датасета
        X_ = X.copy()
        X_.columns = self.rename_cols

        # Временные фичи
        X_['day_sin'] = np.sin(np.pi * 2 * X_['Weekday'] / 7)
        X_['day_cos'] = np.cos(np.pi * 2 * X_['Weekday'] / 7)
        X_['month1_sin'] = np.sin(np.pi * 2 * X_['Month1'] / 12)
        X_['month1_cos'] = np.cos(np.pi * 2 * X_['Month1'] / 12)
        X_['month2_sin'] = np.sin(np.pi * 2 * X_['Month2'] / 12)
        X_['month2_cos'] = np.cos(np.pi * 2 * X_['Month2'] / 12)
        X_['month3_sin'] = np.sin(np.pi * 2 * X_['Month3'] / 12)
        X_['month3_cos'] = np.cos(np.pi * 2 * X_['Month3'] / 12)

        # Категориальные фичи
        X_ = self.bin_encoder.transform(X_)

        features_to_drop = ('Weekday', 'Month1', 'Month2', 'Month3')

        X_ = X_.drop(features_to_drop, axis=1)

        # Масштабирование
        # X_ = self.robust_scaler.transform(X_)

        return X_


In [4]:
cat_features = [
    'Закупочная организация', 'Поставщик',
    'Балансовая единица', 'ЕИ', 'Вариант поставки'
    ]

In [5]:
lgbm_params = {
    'learning_rate': 0.3, 
    'n_estimators': 1000, 
    'max_depth': 8, 
    'max_bin': 128, 
    'num_leaves': 128, 
    'reg_lambda': 0.2,
    }

# Модель
data_preprocessor = DataPreprocessor(cat_features)

estimators = []
for index, params in enumerate(params_list):
    estimator = LGBMClassifier(
        **params,
        n_jobs=-1,
        force_col_wise=True,
        is_unbalance=True
    )
    packed = (f'model_{index}', estimator)
    estimators.append(packed)

model = StackingClassifier(estimators=estimators, n_jobs=-1)

pipeline = Pipeline([
    ('data_preproc', data_preprocessor),
    ('model', model)
])

In [6]:
pipeline.fit(X_train, y_train)
f1_score(y_test, pipeline.predict(X_test), average='macro')

0.9070648717962695

In [ ]:
save_submission(pipeline, 'submission')